# Kubernetes Workloads

In the world of Kubernetes, the term "workload" refers to the heart of your application: the components that you want to run, manage, and scale within the Kubernetes cluster. These workloads could be web servers, databases, microservices, or any other software that you need to deploy and maintain efficiently.

Workloads can be divided into pods and workload resources.

## Pods Overview

- A Kubernetes Pod is the smallest deployable unit, and it serves as the fundamental building block for your workloads. A Pod can contain one or more containers that run together as a single unit.

- It's essential to understand that Pods are designed to be ephemeral. If a Pod fails, Kubernetes can easily recreate it on another node in the cluster. This ephemeral nature ensures resilience and reliability in your applications.

- Kubernetes takes the responsibility of managing and rescheduling Pods in the case of node failures or Pod failures. Cluster administrators typically focus on monitoring and ensuring the overall health of the cluster.

## Workload Resources Overview

In Kubernetes, managing individual Pods manually can be inefficient and error-prone. This is where *Workload Resources* come into play. 

> **Workload Resources** are higher-level abstractions that simplify the deployment, scaling, and lifecycle management of Pods. They provide a declarative approach to managing your application, where you specify what you want, and Kubernetes takes care of the rest.

Here are some common workload resources:

- *Deployment*: Use **Deployments** for deploying stateless applications (applications that do not rely on maintaining persistent, local state or data) that need scaling and rolling updates. Deployments ensure that a specified number of replicas (Pods) are running at all times.

- *Stateful Set*: When you have stateful applications with unique identities (e.g., databases), **Stateful Sets** are the go-to choice. They provide stable network identities and persistent storage.

- *Daemon Set*: Ideal for tasks that require execution on every node within the cluster, such as running logging agents or monitoring tools. They guarantee that exactly one Pod is scheduled to run on each node, ensuring that these essential tasks are distributed across the entire cluster.

- *Job and CronJob*: **Jobs** are used for running batch processes or one-off tasks, while **Cron Jobs** allow you to schedule recurring tasks using a time-based schedule

## Pods

A Pod in Kubernetes is a fundamental concept and serves as the smallest deployable unit. It's essentially a group of one or more containers that share a common context and a specification for running those containers. Think of Pods as similar to `docker-compose`, allowing you to logically group related containers together.

### Shared Context

Pods provide a shared context for containers, which includes:

- **Shared Storage**: Containers within a Pod can share the same storage volume, making it easy for them to exchange data

- **Shared Network Resources**: Containers in a Pod share the same network namespace, often having the same IP address and port space. This allows them to communicate with each other using `localhost`. This sharing of the network namespace is part of how Pods provide network isolation.

- **Linux Namespaces for Isolation**: In addition to sharing the network namespace, Pods use Linux namespaces to isolate containers within the same Pod, providing a level of process and network isolation. These namespaces help ensure that while containers can communicate with each other using `localhost`, they are still effectively isolated from other Pods and resources in the cluster.

Importantly, even within a Pod, individual applications can be further isolated as needed.

### Lifecycle and Pod Phases

Pods in Kubernetes follow a distinct lifecycle, and they have different phases indicating their status and progress. Unlike some container orchestration platforms, Kubernetes Pods are not moved across nodes; instead, they are recreated if necessary. Here are key aspects of the Pod lifecycle:

- **Pod Placement**: Pods remain on the node where they are initially scheduled until they are terminated (either due to a restart policy in case of failures or manual deletion)

- **Pod Recreation**: In Kubernetes, Pods are never moved to different nodes; they are recreated on the same or different nodes if required

- **Self-Healing**: Pods cannot initiate self-healing; they don't attempt to restart themselves. The responsibility for restarting Pods lies with the appropriate Workload Resources (like Deployments or Stateful Sets) or the cluster administrator.

- **Automatic Restart**: Kubernetes relies on the `kubelet` component to automatically restart failed containers within Pods when issues are detected

A Pod can be in one of several phases, each indicating its current status:

- **Pending**: The Pod has been accepted by the Kubernetes cluster, but one or more containers may not have started yet, or the Pod is waiting for node scheduling

- **Running**: At least one container within the Pod is running or being restarted

- **Succeeded**: All containers in the Pod have completed their tasks successfully, and no restart is required

- **Failed**: At least one container has failed and was terminated

- **Unknown**: The state of the Pod cannot be determined, typically due to communication errors with the Node

### Container states

Kubernetes also monitors the state of individual containers within each Pod.

Containers can be in one of three states:

- **Waiting**: This state indicates that the container is waiting for something to happen. It's often seen when a container is downloading its image or pulling secrets required for its operation. 

- **Running**: This state means the container is actively executing its tasks and is in a healthy operational state

- **Terminated**: Containers can be in this state for various reasons. It may be due to a successful termination of the container's task, or it might indicate a failure. When a container terminates, Kubernetes provides the reason for termination and an exit code, which can be invaluable for monitoring and debugging.

### Single-Container Pods

Typically, Pods run with a single container, and a Pod can be seen as a wrapper for that container. Examples include:

- A `FastAPI` server receiving requests and saving data to a shared database
- A Docker container that receives image classification requests and forwards them to a classification service

### Multiple-Container Pods

Multi-container Pods are used in more complex scenarios where multiple tightly coupled containers work together as a cohesive unit. 

<p align=center><img src=images/pod.svg width=350></p>

This setup is ideal for scenarios like:

- Training multiple machine learning models, where containers perform tasks like data transformation, model training, model deployment, and serving
- Implementing a public data-serving container alongside an internal data-writing container with shared storage

> Multi-container Pods are scheduled on the same "logical host" due to their tight coupling.

### Defining Pods

In Kubernetes, you can define Pods imperatively (step by step) or declaratively using `.yaml` files. However, specifying bare Pods directly is generally discouraged because it leaves them without the necessary mechanisms for self-recovery and management. Instead, it's recommended to use higher-level abstractions like Deployments or Replica Sets to manage Pods and ensure their availability.

While it's possible to specify a bare Pod directly, it's important to understand that doing so may not be suitable for most production scenarios. Below is an example of specifying a bare Pod for reference:

``` yaml
apiVersion: v1
kind: Pod
metadata:
  name: pod1
  labels:
    tier: frontend
spec:
  containers:
  - name: hello1
    image: gcr.io/google-samples/hello-app:2.0
```

In this example, we've defined a Pod named `pod1` with a single container called `hello1` using a specific Docker image. However, without higher-level controllers like Deployments or Replica Sets, this Pod won't have the ability to recover from failures or scale automatically.

In practice, for robust and scalable application deployments, it's recommended to use the appropriate Workload Resources, which we'll explore in the next section.

Now it's time to try running some `kubectl` commands yourself. Follow the instructions to run some important Pods commands:

In [2]:
# Observe the pods in the default namespace
!kubectl ###Your command here

NAME         READY   STATUS    RESTARTS   AGE
hello-pod    1/1     Running   0          3h13m
ubuntu-pod   1/1     Running   0          3h6m


In [4]:
# Spin up the pod corresponding to the single-pod configuration above
!kubectl ##Your command here

pod/pod1 created


In [5]:
# Observe the pods in the default namespace again
!kubectl ##Your command here

NAME         READY   STATUS    RESTARTS   AGE
hello-pod    1/1     Running   0          3h14m
pod1         1/1     Running   0          17s
ubuntu-pod   1/1     Running   0          3h6m


In [6]:
# Delete the pod created above
!kubectl ##Your command here

pod "pod1" deleted


In [7]:
# observe the resources in the default namespace again 
!kubectl ##Your command here

NAME         READY   STATUS    RESTARTS   AGE
hello-pod    1/1     Running   0          3h15m
ubuntu-pod   1/1     Running   0          3h7m


Unlike the case in the last notebook, the pod disappears here. This is because, contrary to the case in the last notebook, no instructions were provided here to keep the pod alive. The pod is 'bare', without any resource to keep it running after it fails. 

Keeping it 'alive' is one of the desired states that can be specified in the config file, and it can be achieved with Deployment or Replica Set. We will look at more options in the next section.

## Workload Resources

Before diving into specific Workload Resources, let's cover some important concepts that apply to all of them:

- Each Workload Resource uses the `.spec.template` field to specify how to create a Pod

- The template for a Workload Resource is essentially the same as a Pod configuration, except for the `kind` and `apiVersion` fields

- Workload Resources also have a `.spec.selector` field, which specifies which Pods are managed by the Workload Resource

- The `.spec.selector` can employ matches on defined labels, allowing the Workload Resource to manage Pods even if they are defined in a separate configuration file

Workload Resources can be implemented using different controllers, including `ReplicaSets`, `Deployment`, `DaemonSet`, and `Jobs`. In a future lesson, we will explore another type of Workload Resource called *`StatefulSets`*.

### Replica Sets

>  A `ReplicaSet` is a controller designed to maintain a consistent set of replicated Pods running within a Kubernetes cluster. It ensures that a specified number of identical Pod replicas is continuously available.

#### Acquiring Pods

`ReplicaSet` acquires Pods using the `metadata.ownerReferences` field and matching the `.spec.selector` field. Here's how it works:

- Each Pod has an`metadata.ownerReferences` field that is automatically added by Kubernetes. This field specifies who manages the Pod, which can be another controller.

- If a Pod has no 'owner' (e.g., it's a bare Pod) or its owner is not another controller, and the `.spec.selector` fields match, then the Pod is acquired by the `ReplicaSet`.

For instance, consider a `ReplicaSet` with the following selector in its `.spec.selector`:

```yaml
spec:
  selector:
    matchLabels:
      app: example-app
```
If there is a Pod with the label `app: example-app`, and that Pod does not have another controller owning it, the `ReplicaSet` will acquire and manage that Pod because it matches the specified label selector.

> This acquisition process is similar for other Workload Resources.

#### Using Replica Sets

It is generally recommended to use higher-level controllers like Deployments for managing Pods and Replica Sets. Deployments provide a declarative way to define and manage the desired state of your application, which may involve multiple Replica Sets and Pods. 

Deployments simplify the process of rolling out updates to your application by managing the creation and scaling of Replica Sets, which in turn manage the underlying Pods. However, Replica Sets may be employed when:
- Custom update orchestration is required
- The configuration file will not be updated

For more detailed information on Replica Sets, check [here](https://kubernetes.io/docs/concepts/workloads/controllers/replicaset/).

### Deployment

> A Deployment is a controller in Kubernetes that offers declarative updates for managing Pods and Replica Sets.

Deployments operate on the principle of declarative updates. Instead of specifying the step-by-step process of how to modify an application, you define the desired state, and Kubernetes takes care of the how. They are primarily responsible for controlling the number of replica Pods and the corresponding Replica Sets. They ensure that the desired number of Pods is continuously available while replacing or scaling them as needed.

Key features of Deployments include: 

- **Rolling Updates**: Deployments allow for seamless rolling updates. When you make changes to the desired state (e.g., updating the container image), Deployments gradually replace existing Pods with new ones. This process ensures zero downtime during updates.

- **Rollback Capabilities**: In case of issues during an update, Deployments can easily roll back to a previous version. This feature promotes application reliability.

- **Scaling**: Deployments support both manual and automatic scaling. You can adjust the number of replicas to meet changing demand.

- **Self-Healing**: If a Pod within a Deployment fails, the Deployment controller automatically replaces it to maintain the desired number of replicas

Consider the example config below, and attempt to decipher the meaning of each field:

``` yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  labels:
    app: nginx
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:1.14.2
        ports:
        - containerPort: 80
```

In this example, we instruct the resource to ensure that at least three Pods are running at all times. These Pods are identified using the `selector.matchLabels`, which searches the template for the label with the key `app` and value `nginx`.

The `spec` section defines the desired state for the Deployment:

  - `replicas: 3`: This line specifies that we want to ensure that there are always three replica Pods running for this Deployment. If the actual number of Pods deviates from this, Kubernetes will automatically take corrective actions to meet this desired state.
  - `selector`: The selector field determines which Pods are managed by this Deployment. In this example, the Pods are selected based on the label `app: nginx`, and this label must match the label defined in the `template`.
  - `template`: The `template` section defines the Pod template for the Deployment. This template defines the structure of the Pods, including labels, containers, and other specifications:

    - `metadata`: It provides metadata for the Pods created by this Deployment. Here, we apply the label `app: nginx` to the Pods, matching the label in the `selector`.
    - `spec`: This section describes the Pod's specification, including its containers:
      - `containers`: We define a single container named `nginx` using the official Nginx image (version 1.14.2). This container listens on port 80.

#### Hands-On

1. Create a `.yaml` file with the provided configuration
2. Use the appropriate `kubectl` command to apply the `.yaml` file
3. Observe the number of Pods created
4. Delete one of the Pods
5. Observe the number of Pods again

In [1]:
# Run the .yaml file
!kubectl 

deployment.apps/nginx-deployment created


In [2]:
# Observe how many pods you have
!kubectl

NAME                                READY   STATUS              RESTARTS   AGE
nginx-deployment-66b6c48dd5-7qpvr   0/1     ContainerCreating   0          17s
nginx-deployment-66b6c48dd5-gc7z7   0/1     ContainerCreating   0          17s
nginx-deployment-66b6c48dd5-vzsx2   0/1     ContainerCreating   0          17s


In [1]:
# Delete one of the pods
!kubectl delete pod nginx-deployment-66b6c48dd5-vzsx2

pod "nginx-deployment-66b6c48dd5-vzsx2" deleted


In [2]:
# Observe how many pods you have again
!kubectl get pod

NAME                                READY   STATUS    RESTARTS        AGE
nginx-deployment-66b6c48dd5-7qpvr   1/1     Running   1 (8m15s ago)   8h
nginx-deployment-66b6c48dd5-fsck5   1/1     Running   0               40s
nginx-deployment-66b6c48dd5-gc7z7   1/1     Running   1 (8m15s ago)   8h


After deleting a Pod, you should see that a new Pod has been created by the Deployment to maintain the desired number of replicas.

### Daemon Sets

> A `DaemonSet` is a controller that ensures a Pod is deployed on all Nodes in the cluster. Each Node gets one instance of the Pod.

In the examples thus far, we had one pod per node; therefore, if a node is removed, the number of pods will decrease.

#### Node Selection in Daemon Sets

One distinctive feature of Daemon Sets is that they automatically ensure one pod runs on each node in the cluster. The `replicas` field, commonly used in Deployments, is not present in Daemon Set configurations because there is always one pod per node.

However, there are cases specific to Daemon Sets where you might want to influence the process of node selection:

- Ensuring that a Daemon Set pod runs on a node with specific hardware characteristics (e.g., a node with SSD storage).
- Co-locating Daemon Set pods from different services on the same node if they frequently communicate.

Kubernetes provides a mechanism for node selection through the use of labels and node selectors. Nodes can be labeled with various attributes, and then, in the Daemon Set configuration, you can specify `nodeSelector` rules to determine which nodes receive the Daemon Set pods.

Here's an example `DaemonSet` configuration `YAML`:

``` yaml 
apiVersion: apps/v1
kind: DaemonSet
metadata:
  name: fluentd-elasticsearch
spec:
  selector:
    matchLabels:
      name: fluentd-elasticsearch
  template:
    metadata:
      labels:
        name: fluentd-elasticsearch
    spec:
      containers:
      - name: fluentd-elasticsearch
        image: quay.io/fluentd_elasticsearch/fluentd:v2.5.2
        resources:
          limits:
            memory: 200Mi
          requests:
            cpu: 100m
            memory: 200Mi
      terminationGracePeriodSeconds: 30
```

In this example, a Daemon Set named `fluentd-elasticsearch` ensures that a pod running the specified container image, `quay.io/fluentd_elasticsearch/fluentd:v2.5.2`, is deployed to every node in the cluster.

Before proceeding, let's briefly go over memory resource management in Kubernetes. In the provided Daemon Set configuration, you'll notice a new field within the `template.spec.container` section called `resources`. This field allows you to set resource limits and requests for the container:

- `limits`: These values represent the maximum resource capacities that are allotted to a pod. If a process within the pod consumes more than the specified limit (e.g., exceeding 200MB of RAM), Kubernetes will take action, potentially restarting the pod to enforce resource limits.

- `requests`: These values represent the minimum resource capacities that are guaranteed to be available to the pod. In the example configuration, the pod is guaranteed a minimum of 200MB of RAM and 100 milicores (1/1000th of a core). Kubernetes uses these requests for scheduling and placement decisions.

For more detailed information on container resource management, you can refer to the [official Kubernetes documentation on managing container resources](https://kubernetes.io/docs/concepts/configuration/manage-resources-containers/).

### Jobs

> **Jobs** are a versatile resource designed to create one or more Pods to perform a task and ensure the successful completion of that task. 

Jobs are ideal for orchestrating and executing tasks, which could be one-time or batch processes, within a Kubernetes cluster. The tasks are encapsulated within Pods. They monitor the success of their tasks. They repeatedly attempt to execute Pods until a specified number of them successfully complete the assigned task, ensuring that the job is done correctly. You can specify a *termination policy* for Jobs, which dictates under what conditions a Job is considered successful and can be marked as complete

Jobs enable parallel execution of the same task multiple times. This is especially useful when you want to process a large number of items concurrently, such as data processing, data transformations, or parallel batch jobs.

Here's an example of a Job workload that calculates the value of π:

``` yaml
apiVersion: batch/v1
kind: Job
metadata:
  name: pi
spec:
  template:
    spec:
      containers:
      - name: pi
        image: perl
        command: ["perl",  "-Mbignum=bpi", "-wle", "print bpi(2000)"]
      restartPolicy: Never
  backoffLimit: 4
```

#### Jobs Specification

For Jobs in addition to the standard fields the following fields can also be used: 

- `.spec.restartPolicy`: Can be either `Never` or `OnFailure` (default). It specifies what should happen when a Pod completes its task.

- .`spec.completions`: The number of Pods that should successfully complete before the Job is considered successful

- `.spec.parallelism`: The number of Pods to run simultaneously. This allows you to control parallelism in different ways.

Using the combination of `.spec.completions` and `.spec.parallelism`, you can construct different levels of parallelism:

- **Non-parallel**: If you specify `.spec.completions=1`, only one Job will be created at a time, and a new one will only start after the previous one fails

- **Parallel with a fixed completion count**: By setting `.spec.completions=N`, you can run at most N parallel Jobs at any given time. If a Pod or Job fails, the controller will reschedule it to maintain the desired count.

- **Parallel with a work queue**: This configuration is achieved by specifying `.spec.completions=1` and `.spec.parallelism=N`. In this mode, N Pods will run concurrently, but the execution of additional Pods continues until all tasks are completed. This approach is useful when you want Pods to communicate directly with each other for improved efficiency.

> It's important to note that the default mode is non-parallel, and the default values for `.spec.completions` and `.spec.parallelism `are both set to 1.

Additionally, you can specify the [completion mode](https://kubernetes.io/docs/concepts/workloads/controllers/job/#completion-mode), which allows you to modify the behavior of the Pods upon termination.

Here are some key points to keep in mind:

- Depending on the settings, if `.spec.completions=N` is achieved, the Job is considered successful

- Until that point, Kubernetes will attempt to recreate Pods associated with the Job N times, as specified by `.spec.completions`

- Exponential back-off delay is applied in case of failures, starting with a retry after 10 seconds, then 20 seconds, and doubling each time, capped at a maximum back-off period of 6 minutes

- `.spec.activeDeadlineSeconds`: This field defines the maximum duration (in seconds) that the entire Job is allowed to run. Once this duration is reached, all Pods associated with the Job are terminated, regardless of their completion status.
    
#### Cleaning Up

By default, completed Jobs are not automatically removed from the cluster. This is because you may need to check logs or the status of completed Jobs.

However, to manage resources effectively, you can use *TTL (time to live)* to specify when the Job and its associated Pods should be removed from the cluster. TTL can be set using the `.spec.ttlSecondsAfterFinished` field.

As mentioned, Jobs will terminate when they successfully execute a number of Pods. To repeat that operation, you would need to `apply` a Kubernetes object to re-run the job. Fortunately, there is an easier approach to generate Jobs periodically with the desired frequency, namely *`CronJobs`*.

### Cron Jobs

`CronJobs` create `Jobs` on a recurring schedule. You can specify the schedule in the `spec.schedule` field, similar to a Cron expression.

``` yaml
apiVersion: batch/v1
kind: CronJob
metadata:
  name: hello
spec:
  schedule: "*/1 * * * *"
  jobTemplate:
    spec:
      template:
        spec:
          containers:
          - name: hello
            image: busybox
            imagePullPolicy: IfNotPresent
            command:
            - /bin/sh
            - -c
            - date; echo Hello from the Kubernetes cluster
          restartPolicy: OnFailure
```

This Cron Job is scheduled to run according to the specified Cron schedule, `*/1 * * * *`.

This schedule means that the `CronJob` runs every minute, as indicated by the `*/1 `in the first field, which represents minutes. The `*` in the other fields means "every" for hours, days of the month, months, and days of the week, respectively.

> A `CronJob` specifies its schedule using the Cron syntax, which consists of five fields: minute, hour, day of the month, month, and day of the week. You can use wildcards (`*`) and ranges to define flexible schedules. For example, a `CronJob` that runs every day at midnight would have the schedule `0 0 * * *`.

## Clean Up

Before wrapping up this lesson, let's make sure we clean up all the resources we have been provisioning this lesson. Run the following commands to make sure everything we provision is deleted (you should delete the two `YAML`s you have created, one for the Pod and one for the `nginx` Deployment) using this syntax:

In [ ]:
!kubectl delete -f <your-yaml-file>

## Key Takeaways

- Kubernetes Workloads refer to applications and services running on a Kubernetes cluster. They can be categorized into two main components: Pods and Workload Resources.
- Pods are the smallest deployable units in Kubernetes. They group one or more containers, sharing a common context.
- Workload Resources are higher-level abstractions for managing Pods' lifecycle. They include Replica Sets, Deployments, Daemon Sets, Jobs, and Cron Jobs. Workload Resources simplify the management of Pods, enabling declarative updates and scaling.
- Replica Sets maintain a specified number of replicated Pods. They create and delete Pods based on the desired replica count.
- Deployments provide declarative updates for Pods and Replica Sets. They enable rolling updates and rollbacks, ensuring application availability during changes.
- Daemon Sets ensure that a Pod runs on every node in the cluster. They are suitable for monitoring or managing services on each node.
- Jobs create one or more Pods to complete a task. They can run in parallel, and their completion behavior can be customized.
- Cron Jobs automate the creation of Jobs on a schedule. They allow recurring tasks to be executed at specified intervals.